# Deep-Flow: Vector Field Visualization
Visualizing the learned probability density in the Latent PCA Manifold.

**Axes:**
*   **X:** Principal Component 1 (Usually Speed/Distance)
*   **Y:** Principal Component 2 (Usually Steering/Curvature)
*   **Arrows:** The velocity $v_t$ predicted by the model.

In [1]:
import matplotlib
if not hasattr(matplotlib.RcParams, '_get'):
    matplotlib.RcParams._get = lambda self, key: self.get(key)

import matplotlib.pyplot as plt
import torch
import numpy as np
import json
from omegaconf import OmegaConf
from ipywidgets import interact, FloatSlider

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path: sys.path.append(project_root)

from src.models.deep_flow import DeepFlow
from src.dataset.waymo_dataset import WaymoDataset

In [3]:
# ## 1. Load Model & Data

# %%
cfg = OmegaConf.load("../configs/main_config.yaml")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DeepFlow(cfg).to(device)
checkpoint = torch.load("../checkpoints/best_model.pth", map_location=device)
model.load_state_dict(checkpoint['model'])
model.eval()

# Load stats for reference and PCA basis for visualization
with open("/mnt/d/waymo_datasets/Deep-Flow_Dataset//stats.json", "r") as f: stats = json.load(f)
with open("/mnt/d/waymo_datasets/Deep-Flow_Dataset/pca_basis.json", "r") as f: pca_data = json.load(f)

# Load a scenario
val_set = WaymoDataset(cfg, split='validation', in_memory=True)
idx = 15 # Pick a turning scenario if possible
batch = val_set[idx]
batch_torch = {k: v.unsqueeze(0).to(device) if isinstance(v, torch.Tensor) else v 
               for k, v in batch.items()}

print(f"Scenario: {batch['scenario_id']}")

🚀 Parallel Eager Load (20 workers)...


100%|██████████| 8856/8856 [00:14<00:00, 601.77it/s]

✅ validation split loaded. Count: 8856
Scenario: d932c7648c73548


## 2. Compute the 2D Slice of the Vector Field
We create a grid of points $(c_1, c_2)$ ranging from -3 to +3 standard deviations.
We set components $c_3 \dots c_{12}$ to 0 (the mean).

In [8]:
def compute_field(model, batch, t_value, grid_size=20):
    # 1. Encode Context ONCE
    context, goal_emb = model.encoder(
        batch['agent_context'], batch['agent_mask'], 
        batch['map_context'], batch['map_mask'], batch['goal_pos']
    )
    
    # 2. Create Grid
    # Range: -4 to +4 (Standard Deviations in PCA space)
    x = np.linspace(-10, 10, grid_size)
    y = np.linspace(-10, 10, grid_size)
    X, Y = np.meshgrid(x, y)
    
    # Flatten grid to batch: [N_points, 2]
    grid_flat = np.stack([X.flatten(), Y.flatten()], axis=1)
    N = grid_flat.shape[0]
    
    # 3. Construct Input Latent Vectors [N, 12]
    # Set C1, C2 from grid. Set C3..C12 to 0.0
    latents = torch.zeros(N, 12, device=device)
    latents[:, 0] = torch.from_numpy(grid_flat[:, 0])
    latents[:, 1] = torch.from_numpy(grid_flat[:, 1])
    
    # 4. Prepare Time and Context tiles
    t = torch.ones(N, device=device) * t_value
    ctx_tile = context.repeat(N, 1)
    goal_tile = goal_emb.repeat(N, 1)
    
    # 5. Predict Velocity
    with torch.no_grad():
        v = model.flow_head(latents, t, ctx_tile, goal_tile)
        
    # Extract velocity for C1 and C2
    U = v[:, 0].view(grid_size, grid_size).cpu().numpy()
    V = v[:, 1].view(grid_size, grid_size).cpu().numpy()
    
    return X, Y, U, V


## 3. Interactive Streamline Plot
Slide `t` from 0.0 (Noise) to 1.0 (Data).

In [9]:
def plot_field(t=0.5):
    X, Y, U, V = compute_field(model, batch_torch, t_value=t)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Speed of flow (magnitude)
    speed = np.sqrt(U**2 + V**2)
    
    # Streamplot: Shows the "paths" particles would take
    strm = ax.streamplot(X, Y, U, V, color=speed, cmap='autumn', linewidth=1.5, density=1.5)
    fig.colorbar(strm.lines, label='Flow Magnitude (Correction Strength)')
    
    # Mark the "Attractor" (Where flow goes to 0)
    # This roughly corresponds to the model's prediction for this scenario
    ax.scatter(0, 0, color='black', marker='+', s=100, label='Mean Trajectory')
    
    ax.set_title(f"Learned Vector Field (PC1 vs PC2) @ t={t:.2f}")
    ax.set_xlabel(f"PC1: Speed/Distance (approx) | Var: {pca_data['explained_variance'][0]*100:.1f}%")
    ax.set_ylabel(f"PC2: Steering/Curvature (approx) | Var: {pca_data['explained_variance'][1]*100:.1f}%")
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.grid(True, linestyle='--', alpha=0.3)
    ax.legend()
    
    plt.show()

interact(plot_field, t=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5));

interactive(children=(FloatSlider(value=0.5, description='t', max=1.0), Output()), _dom_classes=('widget-inter…

## 4. Probability Density Contours
Instead of arrows, we visualize the **Probability Mass**.
We sample 1,000 particles from noise ($x_0$) and push them to time $t$.
The resulting density shows the "Confidence Envelope" of the model.

In [15]:
import seaborn as sns
from scipy.stats import gaussian_kde
from ipywidgets import fixed

@torch.no_grad()
def plot_density_contours(model, batch_torch, t_value, pca_data, n_particles=2000):
    # 1. Encode Context
    context, goal_emb = model.encoder(
        batch_torch['agent_context'], batch_torch['agent_mask'], 
        batch_torch['map_context'], batch_torch['map_mask'], batch_torch['goal_pos']
    )
    
    # 2. Sample Particles (x0)
    # We sample a cloud of points in the 12D latent space
    curr_coeffs = torch.randn(n_particles, 12, device=device)
    
    # 3. Integrate to time t (Euler)
    steps = int(t_value * 50) # Scale steps by t
    if steps == 0: steps = 1
    dt = t_value / steps
    
    # Expand context
    ctx_tile = context.repeat(n_particles, 1)
    goal_tile = goal_emb.repeat(n_particles, 1)
    
    for i in range(steps):
        t_tensor = torch.ones(n_particles, device=device) * (i * dt)
        v = model.flow_head(curr_coeffs, t_tensor, ctx_tile, goal_tile)
        curr_coeffs = curr_coeffs + v * dt
        
    # 4. Extract PC1 and PC2 for plotting
    # shape: [N, 12] -> we want [N, 0] and [N, 1]
    data = curr_coeffs.cpu().numpy()
    x = data[:, 0]
    y = data[:, 1]
    
    # 5. Plotting
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # A. Plot the Vector Field Background (Subtle)
    # Re-use previous compute function for background context
    GX, GY, GU, GV = compute_field(model, batch_torch, t_value, grid_size=15)
    ax.streamplot(GX, GY, GU, GV, color='gray', linewidth=0.5, density=1.0)
    
    # B. Plot the Density Contours (KDE)
    # This draws the "Mountain" of probability
    sns.kdeplot(x=x, y=y, fill=True, cmap="viridis", alpha=0.6, levels=10, ax=ax, thresh=0.05)
    
    # C. Plot the Expert Ground Truth (Red Star)
    # We need to project the expert GT into PCA space to see where it falls
    gt_action = batch_torch['target_action'].cpu().numpy()[0] # [12]
    ax.scatter(gt_action[0], gt_action[1], color='red', s=200, marker='*', label='Expert GT', zorder=10)
    
    # D. Labels
    ax.set_title(f"Probability Density Manifold @ t={t_value:.2f}")
    ax.set_xlabel(f"PC1 (Speed) | Expert Value: {gt_action[0]:.2f}")
    ax.set_ylabel(f"PC2 (Steering) | Expert Value: {gt_action[1]:.2f}")
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.legend()
    ax.grid(True, linestyle=':', alpha=0.5)
    
    plt.show()

# Interactive Slider
interact(plot_density_contours, 
         model=fixed(model),           # <--- Use fixed() to pass the model
         batch_torch=fixed(batch_torch), # <--- Use fixed() to pass the data
         pca_data=fixed(pca_data),       # <--- Use fixed() to pass the PCA dict
         t_value=FloatSlider(min=0.1, max=1.0, step=0.1, value=0.9, description="Time (t)"));

interactive(children=(FloatSlider(value=0.9, description='Time (t)', max=1.0, min=0.1), IntSlider(value=2000, …